In [122]:
import os

# Para adquirir o modelo pré-treinado
from fastai.vision.all import *

# Para iterar pelos vídeos e tornar interativo se desejado
import cv2

# Para extrair os rostos dos frames
from facenet_pytorch import MTCNN

# Útil para realizar operações em vetores
import numpy as np

# Dicionario para contabilizar os FAKE e os REAL
from collections import defaultdict

# Limpa prints extras nas células
from IPython.display import clear_output

import pandas as pd

import random

Define-se o dispositivo que irá rodar as computações necessárias

In [2]:
# Definimos um device onde os tensores estarão sendo processados

device = torch.device('cpu')

In [3]:
# Informações para a MTCNN
IMAGE_SIZE = 224
MARGIN = 0
MIN_FACE_SIZE = 90
THRESHOLDS = [0.68, 0.75, 0.80]
POST_PROCESS = False
SELECT_LARGEST = True
KEEP_ALL = False
DEVICE = device

# ----------------------------------

mtcnn = MTCNN(image_size=IMAGE_SIZE,
              margin=MARGIN, 
              min_face_size=MIN_FACE_SIZE, 
              thresholds=THRESHOLDS,
              post_process=POST_PROCESS,
              select_largest=SELECT_LARGEST, 
              keep_all=KEEP_ALL, 
              device=device)

path_to_learner = Path('./models/final_learner.pkl')
learner = load_learner(path_to_learner, cpu=device.type == 'cpu') # As inferências nas imagens serão feitas pela CPU uma vez que será feito vídeo por vídeo

Define-se uma função que extrai os rostos dos vídeos utilizando a MTCNN e os guarda em lista lista.

In [4]:
def extract_faces_from_video(video_path, padding=0, resize_factor=0.6, check_every_frame=30, show_frames=False):
    
    # Captura o vídeo no path
    try:
        cap = cv2.VideoCapture(str(video_path))
    except:
        print("Ocorreu um erro ao carregar o vídeo. Certifique-se de que é um arquivo de vídeo válido.")
        return []
    
    # Pega, em inteiros, a quantidade de frames do vídeo
    v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    faces = []
    for i in range(1, v_len + 1):
        success = cap.grab()
        # Apertar a tecla 'q' para sair do vídeo.
        if not success:
            continue
        if  i % check_every_frame == 0:
            success, frame = cap.retrieve()
            if not success:
                continue
        else:
            continue
        
        if success: # Sucesso na leitura
            # Obtém o frame como PIL Image (ele é capturado no formato BGR porém a MTCNN espera no formato RGB)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb = Image.fromarray(frame_rgb)
            if show_frames:
                frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
                cv2.imshow('frame', frame)

            boxes, _ = mtcnn.detect(frame_rgb) # Detecta as imagens. O método detect só aceita numpy arrays
            
            if boxes is not None: # Só entra se rostos forem detectados
                for box in boxes: # Para cada uma das bouding boxes encontradas em um único frame (a princípio só deve ter uma)
                    box = [int(b) for b in box]
                    # Extrai a face
                    face = frame_rgb.crop(box=(box[0]-padding, 
                                               box[1]-padding, 
                                               box[2]+padding, 
                                               box[3]+padding))
                    faces.append(PILImage(face))

                    if show_frames:
                        frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
                        cv2.imshow('frame', frame)
                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break
                    
                
        else:
            continue
    
    cv2.destroyAllWindows()
    cap.release()
    
    return faces

In [147]:
def display_video_with_detection(video_path, preds, padding=0, resize_factor=0.6, check_every_frame=30):
    
    # Captura o vídeo no path
    try:
        cap = cv2.VideoCapture(str(video_path))
    except:
        print("Ocorreu um erro ao carregar o vídeo. Certifique-se de que é um arquivo de vídeo válido.")
        return []
    
    # Pega, em inteiros, a quantidade de frames do vídeo
    v_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    pred_idx = 0
    
    faces = []
    for i in range(1, v_len + 1):
        ret, frame = cap.read()
        if  i % check_every_frame == 0:
            # Apertar a tecla 'q' para sair do vídeo.
            if not ret:
                continue
        else:
            frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
            cv2.imshow('frame', frame)
            continue
        # Obtém o frame como PIL Image (ele é capturado no formato BGR porém a MTCNN espera no formato RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb = Image.fromarray(frame_rgb)
        
        boxes, _ = mtcnn.detect(frame_rgb) # Detecta as imagens. O método detect só aceita numpy arrays

        if boxes is not None: # Só entra se rostos forem detectados
            for box in boxes: # Para cada uma das bouding boxes encontradas em um único frame (a princípio só deve ter uma)
                box = [int(b) for b in box]
                # Extrai a face
                face = frame_rgb.crop(box=(box[0]-padding, 
                                           box[1]-padding, 
                                           box[2]+padding, 
                                           box[3]+padding))
                faces.append(PILImage(face))
                
                #frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
                coord_1 = (box[0]-padding, box[1]-padding)
                coord_2 = (box[2]+padding, box[3]+padding)
                cv2.rectangle(frame, coord_1, coord_2, (0, 255, 0), 3)
                
                text = "FAKE" if preds[pred_idx] == 0 else "REAL"
                text_color =  (0, 0, 255) if preds[pred_idx] == 0 else (0, 255, 0)
                cv2.putText(frame, text, (box[0], box[1]-7), cv2.FONT_HERSHEY_SIMPLEX, 1.5, text_color, 2, 2)
                pred_idx += 1
              
        frame = cv2.resize(frame, (int(frame.shape[1]*resize_factor), int(frame.shape[0]*resize_factor)))
        cv2.imshow('frame', frame)
                
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


        else:
            continue
    
    cv2.destroyAllWindows()
    cap.release()

In [5]:
def get_predictions(learner, faces, verbose=False):
    predicts = []
    predicts_dict = defaultdict(lambda: 0)
    for i, face in enumerate(faces):
        res = learner.predict(face)
        clear_output()
        if verbose:
            print(f"Predição realizada para a face {i+1}")
        predicts.append(res[1].item())
        predicts_dict[res[0]] += 1
    print('-'*100)
    print(F"Resultados individuais: Quantidade de \033[91m FAKES \033[0m: {predicts_dict['FAKE']} | Quantidade de \033[92m REALS \033[0m: {predicts_dict['REAL']}")
    return predicts

In [6]:
def get_final_prediction_from_predictions(predictions, roh=2.75):
    print("-"*100)
    print(f"Utilizando regra com ρ = {roh}...\n")
    if not isinstance(predictions, np.ndarray):
        predictions = np.array(predictions)
    qtd_fakes = np.count_nonzero(predictions == 0)
    qtd_reals = np.count_nonzero(predictions == 1)
    
    return 'FAKE' if qtd_fakes >= roh*qtd_reals else 'REAL'

### Teste visual rápido

In [117]:
base_path = Path("train_sample_videos")

In [148]:
metadata = pd.read_json(base_path/"metadata.json").transpose()
metadata

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4
...,...,...,...
etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4
etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4
eudeqjhdfd.mp4,REAL,train,None


In [155]:
video_name = random.choice(metadata.index)

In [156]:
check_every_frame = 3
path = base_path/video_name

faces = extract_faces_from_video(path, check_every_frame=check_every_frame)
preds = get_predictions(learner, faces, metadata.loc[video_name].label)

Predição realizada para a face 100
----------------------------------------------------------------------------------------------------
Resultados individuais: Quantidade de  FAKES : 100 | Quantidade de  REALS : 0


In [158]:
display_video_with_detection(path, preds, resize_factor=0.6, check_every_frame=check_every_frame)

---

In [9]:
folder = "test_videos"
list_of_paths = list(Path(folder).glob("*.mp4"))[:10] # 10 primeiros vídeos
list_of_paths[:5]

[Path('test_videos/aassnaulhq.mp4'),
 Path('test_videos/aayfryxljh.mp4'),
 Path('test_videos/acazlolrpz.mp4'),
 Path('test_videos/adohdulfwb.mp4'),
 Path('test_videos/ahjnxtiamx.mp4')]

In [10]:
check_every_frame=30
rho=2.75

tempo_corrido = []
qtde_rostos = []
for path in list_of_paths:
    inicio = time.time()
    
    faces = extract_faces_from_video(path, check_every_frame=check_every_frame)
    if len(faces) == 0:
        print("Não foi possível detectar faces humanas no vídeo fornecido.")
    else:
        preds = get_predictions(learner, faces)
        final_res = get_final_prediction_from_predictions(preds, rho)
    
    final = time.time()

    tempo_corrido.append((final-inicio)) # Pegamos o tempo total e dividimos pela quantidade de faces
    qtde_rostos.append(len(faces))

----------------------------------------------------------------------------------------------------
Resultados individuais: Quantidade de  FAKES : 0 | Quantidade de  REALS : 10
----------------------------------------------------------------------------------------------------
Utilizando regra com ρ = 2.75...



In [43]:
extract_faces_from_video(path, check_every_frame=check_every_frame)

[PILImage mode=RGB size=177x210,
 PILImage mode=RGB size=153x208,
 PILImage mode=RGB size=160x202,
 PILImage mode=RGB size=150x203,
 PILImage mode=RGB size=158x222,
 PILImage mode=RGB size=163x209,
 PILImage mode=RGB size=163x237,
 PILImage mode=RGB size=161x221,
 PILImage mode=RGB size=161x207,
 PILImage mode=RGB size=181x240]

In [11]:
tempo_corrido = np.array(tempo_corrido)
qtde_rostos = np.array(qtde_rostos)

In [12]:
tempo_corrido, qtde_rostos, tempo_corrido*1000/qtde_rostos

(array([3.25596809, 3.27551985, 5.43652892, 3.02479959, 3.08222365,
        5.34700036, 3.03099775, 3.01900196, 2.80803919, 3.20800185]),
 array([ 9, 10, 20, 10, 10, 20, 10, 10,  9, 10]),
 array([361.77423265, 327.55198479, 271.82644606, 302.47995853,
        308.22236538, 267.35001802, 303.09977531, 301.90019608,
        312.00435427, 320.8001852 ]))

In [13]:
tempo_corrido.mean(), tempo_corrido.std()

(3.5488081216812133, 0.9307750851079862)